# Sectoral Report 
**Aim** Create short report on sectoral transition in given country and sector, answering key questions relevant to ASCOR analysts

Input: sector, country

Output: one paragraph answer to key questions

Key questions:

1. Have they set a net zero target? 
2. Have they quantified the level of their targets? 
3. (CHALLENGING) How has the legislation of this country changed in the past 50 years? 
Break into time periods, each evaluated by a different LLM and summarize

Compile into markdown with enforced format

Markdowntemplate

# ASCOR Sectoral Transition Report

## Executive Summary
**Country:** [COUNTRY NAME]  
**Sector:** [SECTOR NAME]  
---

## Key Findings

### 1. Net Zero Target Assessment

**Question:** Have they set a net zero target?

[One paragraph response analyzing whether the specified country has established net zero commitments for the given sector. Include timeline details, scope of coverage, and any conditional aspects of the target.]

**Key Details:**
- Target Date: [DATE or "Not Set"]
- Scope: [Economy-wide/Sectoral/Conditional]
- Legal Status: [Legally Binding/Policy Commitment/Aspirational]

**Confidence in Assessment:** 

### 2. Legislative Evolution Assessment

**Question:** How has the legislation of this country changed in the past 50 years?

[One paragraph response analyzing the evolution of climate and sectoral legislation over the past five decades. Focus on major policy shifts, regulatory frameworks, and legislative milestones relevant to the specified sector.]

**Legislative Timeline:**
- **1970s-1980s:** [Key developments]
- **1990s-2000s:** [Key developments]
- **2010s-Present:** [Key developments]

**Confidence in Assessment:** 


### 0. Setup

In [ ]:
import os
from openai import OpenAI

import os
from openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema import BaseOutputParser
import json
from dotenv import load_dotenv

load_dotenv()


from typing_extensions import Annotated, TypedDict
from typing import Literal

In [ ]:
# Load Models


### 1. Enforce Output schema


In [ ]:
class NetZeroTargetDetails(TypedDict):
    """Structured details for net zero target assessment"""
    target_date: Annotated[str, ..., "Target achievement date or 'Not Set'"]
    scope: Annotated[Literal["Economy-wide", "Sectoral", "Conditional"], ..., "Target scope classification"]
    legal_status: Annotated[Literal["Legally Binding", "Policy Commitment", "Aspirational"], ..., "Legal status of the target"]

class NetZeroAssessment(TypedDict):
    """Complete net zero target assessment from dedicated LLM"""
    net_zero_response: Annotated[str, ..., "One paragraph analyzing net zero commitments, including timeline, scope, and conditional aspects"]
    net_zero_details: Annotated[NetZeroTargetDetails, ..., "Structured key details about the net zero target"]
    net_zero_confidence: Annotated[Literal["High", "Medium", "Low"], ..., "Confidence level in the net zero assessment"]

class LegislativeTimeline(TypedDict):
    """Timeline breakdown of legislative changes from dedicated LLM"""
    period_1970s_1980s: Annotated[str, ..., "Key legislative developments in 1970s-1980s period"]
    period_1990s_2000s: Annotated[str, ..., "Key legislative developments in 1990s-2000s period"]  
    period_2010s_present: Annotated[str, ..., "Key legislative developments in 2010s-Present period"]

class LegislativeSummary(TypedDict):
    """Legislative evolution summary from synthesis LLM"""
    legislation_response: Annotated[str, ..., "One paragraph analyzing 50 years of legislative evolution, focusing on policy shifts and regulatory frameworks"]
    legislation_confidence: Annotated[Literal["High", "Medium", "Low"], ..., "Confidence level in the legislative assessment"]

class SimpleASCORReport(TypedDict):
    """Final assembled report structure"""
    
    # Executive Summary fields
    country: Annotated[str, ..., "Name of the country being analyzed"]
    sector: Annotated[str, ..., "Name of the sector being analyzed"]
    
    # Net Zero Target Assessment (from specialized LLM)
    net_zero_response: Annotated[str, ..., "One paragraph analyzing net zero commitments"]
    net_zero_details: Annotated[NetZeroTargetDetails, ..., "Structured key details about the net zero target"]
    net_zero_confidence: Annotated[Literal["High", "Medium", "Low"], ..., "Confidence level in the net zero assessment"]
    
    # Legislative Evolution Assessment (assembled from multiple LLMs)
    legislation_response: Annotated[str, ..., "One paragraph analyzing 50 years of legislative evolution"]
    legislative_timeline: Annotated[LegislativeTimeline, ..., "Structured timeline of legislative changes by period"]
    legislation_confidence: Annotated[Literal["High", "Medium", "Low"], ..., "Confidence level in the legislative assessment"]


### 2. Get Context

### 2. Prompts

In [ ]:
def generate_net_zero_assessment(country: str, sector: str, net_zero_context: str, net_zero_llm) -> NetZeroAssessment:
    """
    Use specialized LLM with net zero context to generate target assessment
    
    Args:
        country: Country name to analyze
        sector: Sector name to analyze
        net_zero_context: Specific context about net zero targets and commitments
        net_zero_llm: LLM specialized in net zero target analysis
    
    Returns:
        NetZeroAssessment: Structured assessment of net zero targets
    """
    
    structured_net_zero_llm = net_zero_llm.with_structured_output(NetZeroAssessment)
    
    net_zero_prompt = f"""You are an expert in climate policy and net zero target analysis.

ANALYSIS REQUIREMENTS:
- Country: {country}
- Sector: {sector}
- Net Zero Context: {net_zero_context}

TASK: Analyze whether this country has set net zero targets for this sector.

INSTRUCTIONS:
1. Write one detailed paragraph analyzing:
   - Whether net zero commitments exist for this sector
   - Timeline details and target dates
   - Scope of coverage (economy-wide, sectoral, conditional)
   - Any conditional aspects or dependencies

2. Provide structured details:
   - Target date (specific year or "Not Set")
   - Scope classification (Economy-wide/Sectoral/Conditional)
   - Legal status (Legally Binding/Policy Commitment/Aspirational)

3. Assess your confidence level based on available evidence

Base your analysis strictly on the provided context. If information is incomplete, acknowledge this in your confidence assessment."""

    return structured_net_zero_llm.invoke([("human", net_zero_prompt)])

def generate_legislative_timeline(country: str, sector: str, legislative_context: str, timeline_llm) -> LegislativeTimeline:
    """
    Use specialized LLM with legislative context to generate timeline breakdown
    
    Args:
        country: Country name to analyze
        sector: Sector name to analyze  
        legislative_context: Historical legislative context and documents
        timeline_llm: LLM specialized in legislative history analysis
    
    Returns:
        LegislativeTimeline: Structured timeline of legislative changes
    """
    
    structured_timeline_llm = timeline_llm.with_structured_output(LegislativeTimeline)
    
    timeline_prompt = f"""You are an expert in legislative history and policy development.

ANALYSIS REQUIREMENTS:
- Country: {country}
- Sector: {sector}
- Legislative Context: {legislative_context}

TASK: Analyze legislative changes over the past 50 years, broken down by time periods.

INSTRUCTIONS:
For each period, identify and describe key legislative developments:

1. 1970s-1980s: Focus on foundational environmental laws, early climate awareness, basic regulatory frameworks
2. 1990s-2000s: Focus on international commitments, EU integration (if applicable), emissions trading, renewable energy laws
3. 2010s-Present: Focus on climate action laws, net zero legislation, sectoral targets, recent policy updates

For each period, provide:
- Specific law names and dates where available
- Major policy shifts or regulatory changes
- Developments most relevant to the {sector} sector

Base your analysis on the provided legislative context. Be specific about legislation names, dates, and policy mechanisms."""

    return structured_timeline_llm.invoke([("human", timeline_prompt)])

def synthesize_legislative_summary(country: str, sector: str, timeline_data: LegislativeTimeline, synthesis_llm) -> LegislativeSummary:
    """
    Use synthesis LLM to create coherent paragraph from timeline data
    
    Args:
        country: Country name
        sector: Sector name
        timeline_data: Structured timeline from previous step
        synthesis_llm: LLM specialized in synthesis and summarization
    
    Returns:
        LegislativeSummary: One paragraph summary with confidence assessment
    """
    
    structured_synthesis_llm = synthesis_llm.with_structured_output(LegislativeSummary)
    
    synthesis_prompt = f"""You are an expert policy analyst tasked with synthesizing legislative evolution.

COUNTRY: {country}
SECTOR: {sector}

LEGISLATIVE TIMELINE DATA:
- 1970s-1980s: {timeline_data['period_1970s_1980s']}
- 1990s-2000s: {timeline_data['period_1990s_2000s']}  
- 2010s-Present: {timeline_data['period_2010s_present']}

TASK: Write one comprehensive paragraph analyzing the evolution of climate and sectoral legislation over the past five decades.

INSTRUCTIONS:
1. Create a flowing narrative that connects the three time periods
2. Focus on:
   - Major policy shifts and their drivers
   - Evolution of regulatory frameworks
   - Legislative milestones most relevant to the {sector} sector
   - Progression from basic environmental protection to comprehensive climate action

3. Highlight the overall trajectory and key transitions between periods
4. Assess confidence based on the completeness and specificity of the timeline data

The paragraph should read as a coherent analysis, not a list of periods. Show how legislation evolved and built upon previous foundations."""

    return structured_synthesis_llm.invoke([("human", synthesis_prompt)])

### 3. Chaining 

In [ ]:
def generate_multi_llm_ascor_report(
    country: str, 
    sector: str, 
    net_zero_context: str,
    legislative_context: str,
    net_zero_llm,
    timeline_llm, 
    synthesis_llm
) -> tuple[str, SimpleASCORReport]:
    """
    Orchestrate multiple LLMs to generate complete ASCOR report
    
    Args:
        country: Country to analyze
        sector: Sector to analyze
        net_zero_context: Context specific to net zero targets
        legislative_context: Context specific to legislative history
        net_zero_llm: LLM for net zero analysis
        timeline_llm: LLM for legislative timeline analysis
        synthesis_llm: LLM for synthesizing timeline into paragraph
    
    Returns:
        tuple: (markdown_report, structured_data)
    """
    
    # Step 3a: Generate net zero assessment with specialized LLM
    print("Generating net zero assessment...")
    net_zero_assessment = generate_net_zero_assessment(
        country, sector, net_zero_context, net_zero_llm
    )
    
    # Step 3b: Generate legislative timeline with specialized LLM  
    print("Generating legislative timeline...")
    legislative_timeline = generate_legislative_timeline(
        country, sector, legislative_context, timeline_llm
    )
    
    # Step 3c: Synthesize timeline into coherent paragraph
    print("Synthesizing legislative summary...")
    legislative_summary = synthesize_legislative_summary(
        country, sector, legislative_timeline, synthesis_llm
    )
    
    # Step 3d: Assemble final report structure
    final_report = SimpleASCORReport(
        country=country,
        sector=sector,
        net_zero_response=net_zero_assessment['net_zero_response'],
        net_zero_details=net_zero_assessment['net_zero_details'],
        net_zero_confidence=net_zero_assessment['net_zero_confidence'],
        legislation_response=legislative_summary['legislation_response'],
        legislative_timeline=legislative_timeline,
        legislation_confidence=legislative_summary['legislation_confidence']
    )
    
    # Step 3e: Convert to markdown
    markdown_report = convert_to_markdown(final_report)
    
    return markdown_report, final_report

# Step 4: Markdown conversion (unchanged)
def convert_to_markdown(report_data: SimpleASCORReport) -> str:
    """Convert the TypedDict output directly to the required markdown format"""
    
    markdown = f"""# ASCOR Sectoral Transition Report

## Executive Summary
**Country:** {report_data['country']}  
**Sector:** {report_data['sector']}  
---

## Key Findings

### 1. Net Zero Target Assessment

**Question:** Have they set a net zero target?

{report_data['net_zero_response']}

**Key Details:**
- Target Date: {report_data['net_zero_details']['target_date']}
- Scope: {report_data['net_zero_details']['scope']}
- Legal Status: {report_data['net_zero_details']['legal_status']}

**Confidence in Assessment:** {report_data['net_zero_confidence']}

### 2. Legislative Evolution Assessment

**Question:** How has the legislation of this country changed in the past 50 years?

{report_data['legislation_response']}

**Legislative Timeline:**
- **1970s-1980s:** {report_data['legislative_timeline']['period_1970s_1980s']}
- **1990s-2000s:** {report_data['legislative_timeline']['period_1990s_2000s']}
- **2010s-Present:** {report_data['legislative_timeline']['period_2010s_present']}

**Confidence in Assessment:** {report_data['legislation_confidence']}"""
    
    return markdown
